In [4]:
from pantheon import pantheon
import asyncio
import nest_asyncio
nest_asyncio.apply()

server = 'br1'
api_key = 'RGAPI-2a8dfbb0-5d80-4d1b-859d-a3c422ad13a5'

panth = pantheon.Pantheon(server,api_key,True)

In [5]:
async def getSummonerId(name):
    try:
        data = await panth.get_summoner_by_name(name)
        return (data['id'],data['puuid'])
    except Exception as e:
        print(e)

async def getRecentMatchlist(accountId):
    try:
        data = await panth.get_matchlist(accountId, params={"queue":420, 'count':100})
        return data
    except Exception as e:
        print(e)

async def getRecentMatches(accountId):
    try:
        matchlist = await getRecentMatchlist(accountId)
        tasks = [panth.get_match(matchId) for matchId in matchlist]
        return await asyncio.gather(*tasks)
    except Exception as e:
        print(e)

In [7]:
name = "yeo jingoo"

loop = asyncio.get_event_loop()  

(summonerId, accountId) = loop.run_until_complete(getSummonerId(name))
gamesDataRaw = loop.run_until_complete(getRecentMatches(accountId))


Cannot connect to host americas.api.riotgames.com:443 ssl:<ssl.SSLContext object at 0x000001DD252FC7C0> [Connect call failed ('104.160.150.26', 443)]
Cannot connect to host americas.api.riotgames.com:443 ssl:<ssl.SSLContext object at 0x000001DD252FC7C0> [Connect call failed ('104.160.150.26', 443)]
Cannot connect to host americas.api.riotgames.com:443 ssl:<ssl.SSLContext object at 0x000001DD252FC7C0> [Connect call failed ('104.160.150.26', 443)]
Cannot connect to host americas.api.riotgames.com:443 ssl:<ssl.SSLContext object at 0x000001DD252FC7C0> [Connect call failed ('104.160.150.26', 443)]
Cannot connect to host americas.api.riotgames.com:443 ssl:<ssl.SSLContext object at 0x000001DD252FC7C0> [Connect call failed ('104.160.150.26', 443)]


In [8]:
def getParticipantIdFromAccountId(game, accountId):
    for participant in game['info']['participants']:
        if participant['puuid'] == accountId:
            return participant['participantId']

participantId = getParticipantIdFromAccountId(gamesDataRaw[1],accountId)
participantId

2

In [9]:
def getTeamAndChampion(game, participantId):
    for participant in game['info']['participants']:
        if participant['participantId'] == participantId:
            return participant['teamId'], participant['championName'], participant['win']

teamId, championName, win = getTeamAndChampion(gamesDataRaw[1], participantId)
teamId, championName, win

(100, 'Shyvana', False)

In [10]:
def getTeamComposition(game):
    compositions  = {100:[], 200:[]}
    for participant in game['info']['participants']:
        compositions[participant['teamId']].append(participant['championName'])
    return compositions

compositions = getTeamComposition(gamesDataRaw[1])
compositions

{100: ['Volibear', 'Shyvana', 'TwistedFate', 'Kaisa', 'Soraka'],
 200: ['Jax', 'Rengar', 'Veigar', 'Nilah', 'Nami']}

In [11]:
teamComposition = compositions[teamId]
teamComposition = '/'.join(teamComposition)
enemyTeamComposition = compositions[100 if teamId==200 else 200]
enemyTeamComposition = '/'.join(enemyTeamComposition)
teamComposition, enemyTeamComposition

('Volibear/Shyvana/TwistedFate/Kaisa/Soraka', 'Jax/Rengar/Veigar/Nilah/Nami')

In [12]:
def getPatch(game):
    return '.'.join(game['info']['gameVersion'].split('.')[:2])

getPatch(gamesDataRaw[1])

'12.15'

In [13]:
import datetime

def getDate(game):
    return datetime.datetime.fromtimestamp(int(game['info']['gameCreation']/1000)).strftime('%Y/%m/%d')

getDate(gamesDataRaw[1])

'2022/08/10'

In [14]:
def getNeutralObjectives(game):
    neutralObjectives = {
        100:{
            'dragon':{
                'first':None, 
                'kills':None
            },
            'herald':{
                'first':None,
                'kills':None
            },
            'baron':{
                'first':None,
                'kills':None
            }}, 
        200:{
            'dragon':{
                'first':None,
                'kills':None
            },
            'herald':{
                'first':None,
                'kills':None
            },
            'baron':{
                'first':None,
                'kills':None
            }
        }}
    for participant in game['info']['teams']:
        neutralObjectives[participant['teamId']]['dragon']['first'] = participant['objectives']['dragon']['first']
        neutralObjectives[participant['teamId']]['dragon']['kills'] = participant['objectives']['dragon']['kills']
        neutralObjectives[participant['teamId']]['herald']['first'] = participant['objectives']['riftHerald']['first']
        neutralObjectives[participant['teamId']]['herald']['kills'] = participant['objectives']['riftHerald']['kills']
        neutralObjectives[participant['teamId']]['baron']['first'] = participant['objectives']['baron']['first']
        neutralObjectives[participant['teamId']]['baron']['kills'] = participant['objectives']['baron']['kills']
    return neutralObjectives

neutralObjectives = getNeutralObjectives(gamesDataRaw[1])
neutralObjectives

{100: {'dragon': {'first': True, 'kills': 1},
  'herald': {'first': False, 'kills': 1},
  'baron': {'first': True, 'kills': 1}},
 200: {'dragon': {'first': False, 'kills': 2},
  'herald': {'first': True, 'kills': 1},
  'baron': {'first': False, 'kills': 0}}}

In [15]:
def getPosition(game, participantId):
    for participant in game['info']['participants']:
        if participant['participantId'] == participantId:
            return participant['teamPosition']

getPosition(gamesDataRaw[0], 2)

'JUNGLE'

In [16]:
gameInformationList = []

for game in gamesDataRaw:
    participantId = getParticipantIdFromAccountId(game,accountId)
    teamId, championName, win = getTeamAndChampion(game, participantId)
    compositions = getTeamComposition(game)

    gameInformation = {}
    gameInformation['date'] = getDate(game)
    gameInformation['champion'] = championName
    gameInformation['win'] = win
    gameInformation['patch'] = getPatch(game)
    gameInformation['position'] = getPosition(game, participantId)
    gameInformation['teamComposition'] = '/'.join(compositions[teamId])
    gameInformation['enemyTeamComposition'] = '/'.join(compositions[100 if teamId==200 else 200])
    gameInformation['firstDragon'] = getNeutralObjectives(game)[teamId]['dragon']['first']
    gameInformation['firstHerald'] = getNeutralObjectives(game)[teamId]['herald']['first']
    gameInformation['firstBaron'] = getNeutralObjectives(game)[teamId]['baron']['first']
    gameInformation['dragonKills'] = getNeutralObjectives(game)[teamId]['dragon']['kills']
    gameInformation['heraldKills'] = getNeutralObjectives(game)[teamId]['herald']['kills']
    gameInformation['baronKills'] = getNeutralObjectives(game)[teamId]['baron']['kills']
    gameInformation['enemyDragonKills'] = getNeutralObjectives(game)[100 if teamId==200 else 200]['dragon']['kills']
    gameInformation['enemyHeraldKills'] = getNeutralObjectives(game)[100 if teamId==200 else 200]['herald']['kills']
    gameInformation['enemyBaronKills'] = getNeutralObjectives(game)[100 if teamId==200 else 200]['baron']['kills']

    gameInformationList.append(gameInformation)

gameInformationList

[{'date': '2022/08/10',
  'champion': 'Shyvana',
  'win': True,
  'patch': '12.15',
  'position': 'JUNGLE',
  'teamComposition': 'Jayce/Shyvana/Yone/Ezreal/Lux',
  'enemyTeamComposition': 'Olaf/Khazix/Yasuo/Jinx/Karma',
  'firstDragon': False,
  'firstHerald': False,
  'firstBaron': True,
  'dragonKills': 2,
  'heraldKills': 0,
  'baronKills': 1,
  'enemyDragonKills': 2,
  'enemyHeraldKills': 2,
  'enemyBaronKills': 0},
 {'date': '2022/08/10',
  'champion': 'Shyvana',
  'win': False,
  'patch': '12.15',
  'position': 'JUNGLE',
  'teamComposition': 'Volibear/Shyvana/TwistedFate/Kaisa/Soraka',
  'enemyTeamComposition': 'Jax/Rengar/Veigar/Nilah/Nami',
  'firstDragon': True,
  'firstHerald': False,
  'firstBaron': True,
  'dragonKills': 1,
  'heraldKills': 1,
  'baronKills': 1,
  'enemyDragonKills': 2,
  'enemyHeraldKills': 1,
  'enemyBaronKills': 0},
 {'date': '2022/08/10',
  'champion': 'Shyvana',
  'win': True,
  'patch': '12.15',
  'position': 'JUNGLE',
  'teamComposition': 'Sett/Shyva

In [17]:
import pandas as pd
df = pd.DataFrame(gameInformationList)

df

Task exception was never retrieved
future: <Task finished name='Task-1' coro=<getSummonerId() done, defined at C:\Users\julio\AppData\Local\Temp\ipykernel_5724\1548656971.py:1> exception=SystemExit(0)>
Traceback (most recent call last):
  File "c:\Users\julio\AppData\Local\Programs\Python\Python39\lib\site-packages\pantheon\pantheon.py", line 168, in _auto_retry
    return await func(*args, **params)
  File "c:\Users\julio\AppData\Local\Programs\Python\Python39\lib\site-packages\pantheon\pantheon.py", line 237, in _exceptions
    raise exc.Forbidden
pantheon.utils.exceptions.Forbidden: Access forbidden, your key may be expired or blacklisted. Check the dev portal : https://developer.riotgames.com/

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\julio\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_

,date,champion,win,patch,position,teamComposition,enemyTeamComposition,firstDragon,firstHerald,firstBaron,dragonKills,heraldKills,baronKills,enemyDragonKills,enemyHeraldKills,enemyBaronKills
0,2022/08/10,Shyvana,True,12.15,JUNGLE,Jayce/Shyvana/Yone/Ezreal/Lux,Olaf/Khazix/Yasuo/Jinx/Karma,False,False,True,2,0,1,2,2,0
1,2022/08/10,Shyvana,False,12.15,JUNGLE,Volibear/Shyvana/TwistedFate/Kaisa/Soraka,Jax/Rengar/Veigar/Nilah/Nami,True,False,True,1,1,1,2,1,0
2,2022/08/10,Shyvana,True,12.15,JUNGLE,Sett/Shyvana/Yone/Kaisa/Zyra,Teemo/Hecarim/Yasuo/Caitlyn/Senna,False,False,False,0,0,0,0,0,0
3,2022/08/09,Shyvana,True,12.14,JUNGLE,Mordekaiser/Shyvana/Corki/Varus/Zyra,Fiora/Vi/Neeko/Caitlyn/Morgana,True,False,False,2,1,0,0,1,0
4,2022/08/08,Shyvana,False,12.14,JUNGLE,Sett/Shyvana/Leblanc/Twitch/Senna,Urgot/Shaco/Malzahar/Kaisa/Lux,True,False,False,2,0,0,3,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022/05/03,Nasus,False,12.8,TOP,Nasus/Shaco/Katarina/Draven/Morgana,Darius/Nocturne/Sylas/Vayne/Nami,True,False,False,2,0,0,1,2,0
96,2022/05/01,Nasus,False,12.8,TOP,Nasus/Elise/Zyra/Jinx/Lulu,Tryndamere/Warwick/Swain/Kaisa/Seraphine,False,True,True,4,1,1,3,0,2
97,2022/04/30,Nasus,True,12.8,TOP,Nasus/Karthus/Irelia/Lucian/Pyke,Jayce/Sejuani/Ahri/MissFortune/Seraphine,True,True,False,3,1,0,0,0,0
98,2022/04/30,Nasus,True,12.8,TOP,Nasus/Viego/Katarina/Jhin/Alistar,Vayne/Volibear/Irelia/Caitlyn/Xerath,False,False,True,2,0,1,1,1,0


In [18]:
import pygsheets
def write_to_gsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    """
    this function takes data_df and writes it under spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1


In [19]:
service_file_path = 'lol-dashboard-analise-jungle-9e7697b0f9d2.json'
spreadsheet_key = '1GmzplKCIpe05V9Ju4j8mUwWl1gPs_zNMUaVduA1ihws'
sheet_name = 'dados jogos'

write_to_gsheet(service_file_path, spreadsheet_key, sheet_name, df)